### Importar librerias

In [63]:
import pandas as pd
import queue
import numpy as np

### Leer dataset

In [3]:
data = pd.read_csv("Titanic.csv")
data

,Name,PClass,Age,Sex,Survived
0,"Allen, Miss Elisabeth Walton",1st,29.00,female,1
1,"Allison, Miss Helen Loraine",1st,2.00,female,0
2,"Allison, Mr Hudson Joshua Creighton",1st,30.00,male,0
3,"Allison, Mrs Hudson JC (Bessie Waldo Daniels)",1st,25.00,female,0
4,"Allison, Master Hudson Trevor",1st,0.92,male,1
...,...,...,...,...,...
1308,"Zakarian, Mr Artun",3rd,27.00,male,0
1309,"Zakarian, Mr Maprieder",3rd,26.00,male,0
1310,"Zenni, Mr Philip",3rd,22.00,male,0
1311,"Lievens, Mr Rene",3rd,24.00,male,0


### Estructura del árbol

In [5]:
class Arbol:
    def __init__(self, raiz):
        self.hijos = []
        self.raiz = raiz


    def agregarElemento(self, nodo, nodoPadre):
        subarbol = self.buscarSubarbol(nodoPadre)
        subarbol.hijos.append(Arbol(nodo))
    
    
    def buscarSubarbol(self, nodo):
        if self.raiz == nodo:
            return self
        for subarbol in self.hijos:
            arbolBuscado = subarbol.buscarSubarbol(nodo)
            if (arbolBuscado != None):
                return arbolBuscado
        return None  

    
    def profundidad(self):
        if len(self.hijos) == 0: 
            return 1
        else:
            maxprof = 0;
            for subarbol in self.hijos:
                prof = subarbol.profundidad()
                if prof > maxprof:
                    maxprof = prof
            return maxprof + 1
    
    
    def grado(self, num):
        num += 1
        for hijo in self.hijos:
            grado(hijo, num)
    
    
    def printArbolProfundidad(self, level = 0):
        print("|" + "-" * level * 20 + " " + self.raiz)
        for hijo in self.hijos:
            hijo.printArbolProfundidad(level+1)
    
    
    def printArbolAnchura(self, cola = queue.Queue()):
        print(self.raiz)
        if(len(self.hijos) > 0):
            for hijo in self.hijos:
                cola.put(hijo)
        if(cola.qsize() != 0):
            cola.get().printArbolAnchura(cola)

In [6]:
abuela = "Jacqueline Gurney"
marge = "Marge Bouvier"
patty = "Patty Bouvier"
selma = "Selma Bouvier"
bart = "Bart Simpson"
lisa = "Lisa Simpson"
maggie = "Maggie Simpson"
ling = "Ling Bouvier"

arbol = Arbol(abuela)
arbol.agregarElemento(patty, abuela)
arbol.agregarElemento(selma, abuela)
arbol.agregarElemento(ling, selma)
arbol.agregarElemento(marge, abuela)
arbol.agregarElemento(bart, marge)
arbol.agregarElemento(lisa, marge)
arbol.agregarElemento(maggie, marge)

print(arbol.profundidad())
arbol.printArbolProfundidad()
arbol.printArbolAnchura()


3
| Jacqueline Gurney
|-------------------- Patty Bouvier
|-------------------- Selma Bouvier
|---------------------------------------- Ling Bouvier
|-------------------- Marge Bouvier
|---------------------------------------- Bart Simpson
|---------------------------------------- Lisa Simpson
|---------------------------------------- Maggie Simpson
Jacqueline Gurney
Patty Bouvier
Selma Bouvier
Marge Bouvier
Ling Bouvier
Bart Simpson
Lisa Simpson
Maggie Simpson


### ID3

In [130]:
def id3(df, columna, arbol = None):
    ncol = len(df.columns)        ## Numero de atributos
    ganancias = []                ## Aqui guardo las ganancias
    
    col_class = df[columna]     ## Esta es la columna a predecir
    etiqueta_class = col_class.value_counts();   ## Numero de elementos de cada tipo en la columna a predecir
    entropy_class = np.array(etiqueta_class) / len(col_class.index)  
    entropy_class = -np.sum(entropy_class)  ## entropia de la columna a predecir
    
    for i in range(ncol):   ## Por cada Atributo
        
        col = df.iloc[:,i]  ## Cojo la columna
        if col.name != columna: ## Si no es la que queremos predecir
            semicalculo = []    ## No hagas mucho caso a esto por ahora
            numero_etiquetas = df.iloc[:,i].value_counts()  ## Esto es el numero de elementos de cada tipo en la columna seleccionada
            etiquetas = numero_etiquetas.index ## Esto son los distintos elementos, los nombres
            class_values = data[columna].dropna().unique() ## Nombre de los elementos de la columna a predecir
            
            for j in etiquetas:  ## Por cada elemento distinto en la columna
                number = []  ## Variable auxiliar
                for z in class_values:  ## Por cada elemento distinto en la columna a predecir
                    number.append(len(data[(data[[col.name]] == j) & (data[[columna]] == z) ].index)) ## Los cuento
                
                number = np.array(number) / numero_etiquetas[j]  
                number = number * np.log(number)
                entropia = -np.sum(number)  ## Entropia
                semicalculo.append(numero_etiquetas[j] / len(col_class.index) * entropia) ## Aqui guardo un precalculo de la ganancia
                #print(entropia)
            
            ganancias.append(entropy_class - np.sum(semicalculo))  ## ganancia
            
    return ganancias
   
#print(data)
id3(data.iloc[1:5], 'Survived')

Allison, Master Hudson Trevor                    1
Allison, Mr Hudson Joshua Creighton              1
Allison, Miss Helen Loraine                      1
Allison, Mrs Hudson JC (Bessie Waldo Daniels)    1
Name: Name, dtype: int64
1st    4
Name: PClass, dtype: int64
0.92     1
25.00    1
30.00    1
2.00     1
Name: Age, dtype: int64
female    2
male      2
Name: Sex, dtype: int64


[18853.863489919142, 3802.6136244045774, 18853.863489919142, 8516.329496884362]